<a href="https://colab.research.google.com/github/ImajiPietersz/FYP/blob/main/CWRU_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"imajipietersz","key":"a867e2513f7abed17b6257011d4c5467"}'}

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#change the permission
!chmod 600  ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d brjapon/cwru-bearing-datasets

 64% 26.0M/40.4M [00:00<00:00, 78.8MB/s]
100% 40.4M/40.4M [00:00<00:00, 97.3MB/s]


In [10]:
from zipfile import ZipFile
file_name = "/content/cwru-bearing-datasets.zip"

with ZipFile(file_name, 'r') as zip1:
  zip1.extractall()
  print('Done')

Done


In [11]:
import matplotlib.pyplot as plt
import scipy.io
import numpy as np
import pandas as pd
import os

In [12]:
import matplotlib.pyplot as plt

In [13]:
df = pd.read_csv('/content/feature_time_48k_2048_load_1.csv')

In [14]:
df

,max,min,mean,sd,rms,skewness,kurtosis,crest,form,fault
0,0.35986,-0.41890,0.017840,0.122746,0.124006,-0.118571,-0.042219,2.901946,6.950855,Ball_007_1
1,0.46772,-0.36111,0.022255,0.132488,0.134312,0.174699,-0.081548,3.482334,6.035202,Ball_007_1
2,0.46855,-0.43809,0.020470,0.149651,0.151008,0.040339,-0.274069,3.102819,7.376926,Ball_007_1
3,0.58475,-0.54303,0.020960,0.157067,0.158422,-0.023266,0.134692,3.691097,7.558387,Ball_007_1
4,0.44685,-0.57891,0.022167,0.138189,0.139922,-0.081534,0.402783,3.193561,6.312085,Ball_007_1
...,...,...,...,...,...,...,...,...,...,...
2295,0.21425,-0.19839,0.010769,0.064100,0.064983,-0.212497,-0.119312,3.297037,6.034174,Normal_1
2296,0.21967,-0.20882,0.013136,0.068654,0.069883,-0.061308,-0.295122,3.143410,5.319958,Normal_1
2297,0.20799,-0.21613,0.012571,0.067128,0.068279,-0.154754,-0.071405,3.046161,5.431299,Normal_1
2298,0.21425,-0.22405,0.012608,0.066813,0.067977,-0.326966,0.023662,3.151821,5.391672,Normal_1


In [15]:
df['fault'].value_counts()

Ball_007_1    230
Ball_014_1    230
Ball_021_1    230
IR_007_1      230
IR_014_1      230
IR_021_1      230
OR_007_6_1    230
OR_014_6_1    230
OR_021_6_1    230
Normal_1      230
Name: fault, dtype: int64

In [16]:
labels=df.pop('fault')

In [17]:
labels

0       Ball_007_1
1       Ball_007_1
2       Ball_007_1
3       Ball_007_1
4       Ball_007_1
           ...    
2295      Normal_1
2296      Normal_1
2297      Normal_1
2298      Normal_1
2299      Normal_1
Name: fault, Length: 2300, dtype: object

In [18]:
binary_labels=[]
for i in labels:
  if i=='Normal_1':
    binary_labels.append(1)
  else:
    binary_labels.append(-1)

In [19]:
df['binary_labels']=binary_labels

In [20]:
normal_df = df[df['binary_labels'] == 1]

In [21]:
anomaly_df = df[df['binary_labels'] == -1]

In [22]:
normal_df.pop('binary_labels')

2070    1
2071    1
2072    1
2073    1
2074    1
       ..
2295    1
2296    1
2297    1
2298    1
2299    1
Name: binary_labels, Length: 230, dtype: int64

In [23]:
anomaly_df.pop('binary_labels')

0      -1
1      -1
2      -1
3      -1
4      -1
       ..
2065   -1
2066   -1
2067   -1
2068   -1
2069   -1
Name: binary_labels, Length: 2070, dtype: int64

In [ ]:
len(anomaly_df)

2070

In [24]:
from sklearn import svm
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
X=normal_df
X_train, X_test,_ = np.split(X, [round(len(X) * 0.7), len(X)])
X_normal=normal_df


clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(X_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_normal)
n_error_train = y_pred_train[y_pred_train == -1].size
n_error_test = y_pred_test[y_pred_test == -1].size
n_error_outliers = y_pred_outliers[y_pred_outliers == 1].size




In [25]:
n_error_train

17

In [26]:
n_error_test

2

In [29]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score

def splitTrainTest(ratio, data, labels):
  
  x_train, x_test,_ = np.split(data, [round(len(data) * ratio), len(data)])
  labels_train, labels_test,_ = np.split(labels, [round(len(labels) * ratio), len(labels)])

  return (x_train,labels_train, x_test,labels_test)

# Split the dataset into training and testing sets
X_train, y_train, X_test, y_test = splitTrainTest(0.7,df, binary_labels)
# Create an instance of the OneClassSVM class
ocs = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)

# Fit the model to the training data
ocs.fit(X_train)

# Predict the labels of the testing data
y_pred = ocs.predict(X_test)
y_test = pd.Series(np.array(y_test)).replace({"Good": 1, "Faulty": -1})

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.50      0.51      0.51       460
           1       0.00      0.00      0.00       230

    accuracy                           0.34       690
   macro avg       0.25      0.25      0.25       690
weighted avg       0.34      0.34      0.34       690

